This notebook explores ways to access CHARTEVENTS data. The primary challenge faced is the high dimensionality of this table, with the dataset being 35gb in size. 

In [12]:
import pandas as pd
from datetime import timedelta

In [7]:

chunk_size = 50000 
chunks = []

chartevents_path = '../data/CHARTEVENTS.csv'

i=0
for chunk in pd.read_csv(chartevents_path, chunksize=chunk_size):
    chunks.append(chunk)
    i=i+1
    print(i)
    if i == 1:
      break
    
# if ITEMID in [618, 220210, 3603, 224689, 614, 651, 224422, 615, 224690]:
#   return respiratory_rate


# Concatenate back to df
# df = pd.concat(chunks, axis=0)

1


In [10]:
df = pd.DataFrame(chunks[0])
df

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,788,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525,15.00,15.00,L/min,0,0,NaN,NaN
1,789,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525,100.00,100.00,NaN,0,0,NaN,NaN
2,790,36,165660,241249.0,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823,0.37,0.37,NaN,0,0,NaN,NaN
3,791,36,165660,241249.0,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,6.00,6.00,min,0,0,NaN,NaN
4,792,36,165660,241249.0,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823,2.50,2.50,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,51680,384,122988,279422.0,220546,2163-03-10 04:38:00,2163-03-10 05:18:00,20889,9.50,9.50,K/uL,0,0,NaN,NaN
49996,51681,384,122988,279422.0,220602,2163-03-10 04:38:00,2163-03-10 06:16:00,20889,93.00,93.00,mEq/L,1,0,NaN,NaN
49997,51682,384,122988,279422.0,220615,2163-03-10 04:38:00,2163-03-10 06:16:00,20889,8.50,8.50,mg/dL,1,0,NaN,NaN
49998,51683,384,122988,279422.0,220621,2163-03-10 04:38:00,2163-03-10 06:16:00,20889,137.00,137.00,mg/dL,1,0,NaN,NaN


In [13]:

# Assume df is your DataFrame
df['CHARTTIME'] = pd.to_datetime(df['CHARTTIME'])  # convert CHARTTIME to datetime

# Group by HADM_ID and find the minimum CHARTTIME for each group
min_charttimes = df.groupby('HADM_ID')['CHARTTIME'].min().reset_index()

# Merge min_charttimes back to the original DataFrame
df = df.merge(min_charttimes, on='HADM_ID', suffixes=('', '_MIN'))

# Filter rows where CHARTTIME is within 4 hours of the first CHARTTIME
mask = (df['CHARTTIME'] <= df['CHARTTIME_MIN'] + timedelta(hours=4))

# Apply the mask to select only the desired rows
result = df.loc[mask]

# Drop the added CHARTTIME_MIN column
result = result.drop(columns=['CHARTTIME_MIN'])

ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.